In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import callbacks
from keras.models import Sequential

import matplotlib.pyplot as plt
from sklearn import metrics
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

from tensorflow.keras.layers import BatchNormalization

import keras
from keras import *
from keras.layers import *
from pythainlp import word_tokenize
from pythainlp.word_vector import *
from tensorflow.keras import regularizers
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

from pythainlp import word_vector

import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', 
    patience=8, 
    min_delta=0.001, 
    mode='max'
)

In [2]:
df = pd.read_csv('main_suicidal_data.csv').drop(columns='Unnamed: 0')

In [3]:
wModel = word_vector.WordVector(model_name="thai2fit_wv").get_model()
thai2dict = {}
for word in wModel.index_to_key:
    thai2dict[word] = wModel[word]
thai2vec = pd.DataFrame.from_dict(thai2dict,orient='index')
thVocab = thai2vec.index.to_list()

In [4]:
ll = len(thai2vec)
for vidx in range(ll):
    if vidx % 100 == 0:
        print('\r' + str(vidx),end='')
    aa = thai2vec.iloc[[vidx]]
    ab = aa.values.tolist()
    if vidx == 0:
        vect = ab
    else:
        vect = np.vstack((vect,ab))

print("\n", vect.shape)

51300
 (51358, 300)


In [5]:
def tokenWord(wordTarget):
    wordToken = word_tokenize(wordTarget, engine='attacut')
    return wordToken

def convWord(cw):
    cWord = cw
    for ti in range(len(cWord)):
        if cWord[ti] == ' ':
            cWord[ti] = ''
        elif cWord[ti] not in thVocab:
            cWord[ti] = ''
    return cWord

def token2index(t2idx):
    w2index = []
    for wi in range(len(t2idx)):
        if t2idx[wi] in thVocab:
            w2index.append(thVocab.index(t2idx[wi]))
    return np.array(w2index)

def findMaxArray(fma):
    maxlen = 0
    for mi in range(len(fma)):
        nA = len(fma[mi])
        if nA > maxlen:
            maxlen = nA
    return maxlen

def fill0in(f0i):
    fMax = findMaxArray(f0i)
    for fi, ax in enumerate(f0i):
        if len(ax) < fMax:
            f0i[fi] = np.hstack((ax , np.zeros(fMax-len(ax))))
        f0i1 = np.array(f0i)
    return f0i1

def prepare2train(ipt):
    pre2t = []
    for pidx in range(len(ipt)):
        wp1 = ipt[pidx]
        pre2t.append(token2index(convWord(tokenWord(wp1))))
    return pre2t

In [6]:
X = df['Tweet']
X_arr = X.to_list()
y = df['Label (Specialist)']

In [7]:
le = LabelEncoder()
le.fit(y)

LabelEncoder()

In [8]:
X = prepare2train(X_arr)
y = le.transform(y)

In [9]:
y = np.array(y)
X = fill0in(X)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# LSTM

In [38]:
model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim=51358,output_dim=300,name='embed'))

#Layer 2: LSTM layer
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Dropout(rate=0.8))
#Layer 3: LSTM layer
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(rate=0.8))

#Layer 4: Dense layer (Hidden layer)
model.add(Dense(24, activation = 'relu'))
model.add(Dropout(rate=0.5))

#Layer 5: Output layer
model.add(Dense(6, activation = 'softmax'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed (Embedding)           (None, None, 300)         15407400  
                                                                 
 bidirectional_22 (Bidirecti  (None, None, 128)        186880    
 onal)                                                           
                                                                 
 dropout_23 (Dropout)        (None, None, 128)         0         
                                                                 
 bidirectional_23 (Bidirecti  (None, 64)               41216     
 onal)                                                           
                                                                 
 dropout_24 (Dropout)        (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 24)               

In [39]:
from tensorflow.keras.optimizers import SGD
opt = SGD(learning_rate=0.001,momentum=0.9,nesterov=True)
adam_opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [40]:
model.get_layer('embed').set_weights([vect])
model.get_layer('embed').trainable = True


In [41]:
history = model.fit(X_train,y_train,epochs=20,batch_size=32,callbacks=[early_stopping],validation_split=0.1)

Epoch 1/20


InvalidArgumentError: Cannot assign a device for operation sequential_9/embed/embedding_lookup: Could not satisfy explicit device specification '' because the node {{colocation_node sequential_9/embed/embedding_lookup}} was colocated with a group of nodes that required incompatible device '/job:localhost/replica:0/task:0/device:GPU:0'. All available devices [/job:localhost/replica:0/task:0/device:CPU:0, /job:localhost/replica:0/task:0/device:GPU:0]. 
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=2 requested_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' assigned_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' resource_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
UnsortedSegmentSum: GPU CPU 
StridedSlice: GPU CPU 
Const: GPU CPU 
Shape: GPU CPU 
_Arg: GPU CPU 
Unique: GPU CPU 
Identity: GPU CPU 
ResourceSparseApplyKe

In [ ]:
history = model.fit(X_train,y_train,epochs=20,batch_size=32,callbacks=[early_stopping],validation_split=0.1)

Epoch 1/20


InvalidArgumentError: Cannot assign a device for operation sequential_9/embed/embedding_lookup: Could not satisfy explicit device specification '' because the node {{colocation_node sequential_9/embed/embedding_lookup}} was colocated with a group of nodes that required incompatible device '/job:localhost/replica:0/task:0/device:GPU:0'. All available devices [/job:localhost/replica:0/task:0/device:CPU:0, /job:localhost/replica:0/task:0/device:GPU:0]. 
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=2 requested_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' assigned_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' resource_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
UnsortedSegmentSum: GPU CPU 
StridedSlice: GPU CPU 
Const: GPU CPU 
Shape: GPU CPU 
_Arg: GPU CPU 
Unique: GPU CPU 
Identity: GPU CPU 
ResourceSparseApplyKe

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics)
    plt.plot(epochs, val_metrics)
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()


In [ ]:
plot_metric(history, 'loss')

In [ ]:
model_acc = model.evaluate(X_test,y_test)

In [ ]:
y_pred = np.argmax(model.predict(X_test),axis=1)

In [ ]:
print(classification_report(y_test, y_pred))